### ※ 정책 그레이디언트 : 높은 보상을 얻는 방향의 그레이디언트를 따르도록 정책의 파라미터를 최적화하는 알고리즘

### ✔ 가장 많이 사용하는 알고리즘 : Reinforce 알고리즘
        1. 신경망 정책이 일단 여러번 게임을 해보고, 매 스텝마다 선택된 행동이 더 높은 가능성을 가지도록 만드는 그레이디언트 계산
        2. 각 행동의 이익을 계산
        3. 그 이익이 양수면 계산한 그레이디언트 적용 / 음수면 그것과 반대의 그레이디언트 적용
        4. 마지막으로 모든 결과 그레이디언트 벡터를 평균 내어서 경사 하강법 스텝 적용
           → Trial - Error 방식

In [1]:
import gym
env = gym.make("CartPole-v1")
obs = env.reset()
obs

array([ 0.00362693, -0.02867254, -0.00036954,  0.02584139], dtype=float32)

### 1. 한 스텝을 진행할 함수 만들기

In [2]:
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow import keras
import numpy as np

n_inputs = 4

model = keras.models.Sequential([
    keras.layers.Dense(5, activation="elu", input_shape=[n_inputs]),
    keras.layers.Dense(1, activation="sigmoid"),
])

c:\users\ativ\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ativ\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ativ\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ativ\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [3]:
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        #GradientTape 다들 기억나시죠? (12장) 주어진 입력 변수에 대한 연산의 Gradient를 계산하고 tape에 기록하는 역할을 합니다
        left_proba = model(obs[np.newaxis])
        action = (tf.random.uniform([1,1]) > left_proba) #랜덤한 실수 [1,1] 배열을 하나 만들고 left_proba와 비교 -> True/False로 표현
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32) #왼쪽으로 이동할 타깃 확률 : 1 - (실수로 변환된 행동)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba)) #손실함수를 사용해서 손실 계산, 테이프를 사용해서 손실 그레디언트 계산
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(int(action[0,0].numpy()))
    return obs, reward, done, grads # 플레이 한 후 관측, 보상, 에피소드 종료여부, 계산된 그레이디언트 반환

### 2. 여러 에피소드를  플레이한 후 전체 보상과 각 에피소드, 스텝의 그레이디언트 반환하는 함수 만들기

In [4]:
def play_multiple_episodes(evn, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        obs = env.reset()
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        all_rewards.append(current_rewards) #플레이한 후 보상과 그레이디언트를 append
        all_grads.append(current_grads)
    return all_rewards, all_grads #리스트 반환

### 3. 미래 보상 할인 후 합계 함수 / 정규화용 함수

In [5]:
def discount_rewards(rewards, discount_factor): 
    discounted = np.array(rewards)
    for step in range(len(rewards)-2, -1, -1):
        discounted[step] += discounted[step+1] * discount_factor #할인된 미래 보상의 합 계산
    return discounted

In [6]:
def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(rewards, discount_factor)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards) #다 붙인 다음
    reward_mean = flat_rewards.mean() #평균
    reward_std = flat_rewards.std() #표준편차
    return [(discounted_rewards - reward_mean) / reward_std #정규화
           for discounted_rewards in all_discounted_rewards]

In [7]:
discount_rewards([10, 0, -50], discount_factor=0.8) #해당 스텝 기준으로 미래 보상을 보여줌

array([-22, -40, -50])

In [8]:
discount_and_normalize_rewards([[10, 0, -50], [10, 20]], discount_factor = 0.8)
#위의 모든 step에 대해 미래 보상을 계산한 후 평균을 빼고 '모'표준편차를 구함
#첫번째 출력은 다 음수니까 좋지 않은 행동
#두번째 출력은 다 양수니까 좋은 행동

[array([-0.28435071, -0.86597718, -1.18910299]),
 array([1.26665318, 1.0727777 ])]

### 4. 알고리즘 실행해보기

In [9]:
n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_factor = 0.95
#훈련 반복 : 150번 // 각 반복마다 에피소드 10번 실행 // 각 에피소드마다 스텝을 최대 200번 실행 // 할인계수 0.95
optimizer = keras.optimizers.Adam(lr=0.01)
loss_fn = keras.losses.binary_crossentropy
#무난한 Adam 옵티마이저 사용 // 이진 분류기 훈련이므로 이진 크로스 엔트로피 손실 함수 사용

In [ ]:
for iteration in range(n_iterations):
    all_rewards, all_grads = play_multiple_episodes(
        env, n_episodes_per_update, n_max_steps, model, loss_fn) # 10번 플레이!
    all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_factor) #각 행동의 정규화 이익 계산
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
             [final_reward * all_grads[episode_index][step][var_index]
             for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis = 0)
        all_mean_grads.append(mean_grads)
        #훈련가능한 변수들에 대해 그레이디언트를 final_reward로 가중치를 두어 평균
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables)) #평균 그레이디언트를 옵티마이저에 적용, 훈련 변수 변경